In [7]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import utils.satellite_util  as ut
from machine_learning.logistic_regression_utils import extract_uganda_features, load_satellite_labels, load_satellite_labels_test,extract_uganda_features_test, get_locs

In [8]:
f = h5py.File('data/assorted_images/satellite_images.h5','r')
print('Keys: %s' % f.keys())
a_group_key = list(f.keys())[0]
print(f['data'].shape)
labels = load_satellite_labels()
feats = extract_uganda_features()
print(labels.shape)
print(feats.shape)



Keys: <KeysViewHDF5 ['data', 'labels']>
(641, 224, 224, 3)
(512, 1)
(512, 4097)


In [10]:
coordinates = ut.get_coordinate_list()
#print(coordinates)
lat = []
long = []

for row in coordinates:
    lat.append(row[0])
    long.append(row[1])
    ut.download_img(row[0], row[1], save=True)

Saving: img_2.jpg
Saving: img_3.jpg
Saving: img_4.jpg
Saving: img_5.jpg


KeyboardInterrupt: 

In [ ]:
data_original = feats
print(data_original)

In [ ]:
# train_ratio, val_ratio = 0.9, 0.05
# X_train = data_original[:int(data_original.shape[0]*train_ratio), ...] # ... means all the other axes
# y_train = labels[:int(data_original.shape[0]*train_ratio), ...]

# X_val = data_original[int(data_original.shape[0]*train_ratio):int(data_original.shape[0]*(train_ratio+val_ratio)), ...]
# y_val = labels[int(data_original.shape[0]*train_ratio):int(data_original.shape[0]*(train_ratio+val_ratio)), ...]

# X_test = data_original[int(data_original.shape[0]*(train_ratio+val_ratio)):, ...]
# y_test = labels[int(data_original.shape[0]*(train_ratio+val_ratio)):, ...]

val_labels = load_satellite_labels_test()
val_ex = extract_uganda_features_test()
print(val_labels.shape)
print(val_ex.shape)

In [ ]:
#train_data = data_original[:int(len(data_original)*0.9)]
#train_labels = labels[:int(len(data_original)*0.9)]
#val_data = data_original[int(len(data_original)*0.9):]

In [ ]:
import keras
from keras.models import Sequential # Sequential is one of the main models in Keras, which is basically a sequentially stacked series of layers

model = Sequential() # Initialize a Sequential model instance

In [ ]:
class_weight = {0:1.,
               1:3.}
from keras.layers import Dense
#model.add(Dense(units=512, activation='relu', input_dim=4097)) 
model.add(Dense(units=216, activation='relu', input_dim=96488448))
model.add(keras.layers.Dropout(0.5, noise_shape=None, seed=None))
#model.add()
#model.add(Dense(units=64,activation='sigmoid',input_dim=224*224*3))
# model.add(Dense(units=64,activation='relu',input_dim=512))
#keras.layers.Dropout(0.7, noise_shape=None, seed=None)
#model.add(Dense(units=32,activation='sigmoid',input_dim=224*224*3))
#model.add(Dense(units=32,activation='relu',input_dim=512))
model.add(keras.layers.Dropout(0.5, noise_shape=None, seed=None))
#model.add(Dense(units=16, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid',input_dim = 216))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.SGD(lr=0.001))

In [ ]:
# X_train_flat = X_train.reshape((-1, 224*224*3)) # -1 means letting NumPy to figure this axis out automatically
# X_val_flat = X_val.reshape((-1, 224*224*3))
# X_test_flat = X_test.reshape((-1, 224*224*3))

# print(X_train_flat.shape)
# print(X_val_flat.shape)
# print(X_test_flat.shape)

# Then use fit() to actually train our model
# epochs is basically how many iterations we want for the update process. The model needs some time to reach the optimal state!
# batch_size is how many images we use each time to estimate the gradient. Remember that the more we use the more accurate each update will be, but it will also be slower
#print(labels)
from collections import Counter
training_data = data_original[:490]
print(val_ex.shape,data_original.shape, val_labels.shape, labels.shape)
combined_val_data = np.append(val_ex, data_original[490:], axis=0)
combined_val_labels = np.append(val_labels, labels[490:], axis=0)
print(combined_val_data.shape)
print(combined_val_labels.shape)


print(Counter(val_labels.squeeze()))
print(Counter(labels[:490].squeeze()))
print(np.mean(val_ex),np.mean(data_original))

#model.fit(data_original, labels, validation_data=(val_labels,val_ex),epochs=400, batch_size=3128, shuffle=True,class_weight=class_weight)
model.fit(training_data, labels[:490], validation_data=(combined_val_data, combined_val_labels), epochs=50, batch_size=256, shuffle=True,class_weight=class_weight)


In [ ]:
ac = model.evaluate(feats,labels)
print('The train accuracy is:{}'.format(ac))

acc = model.evaluate(val_ex, val_labels)
print('The val accuracy is: {}'.format(acc))
# fl = np.array(f['data'])
# flat = np.ndarray.flatten(fl)
# print(flat.shape)
# And make predictions
prob = model.predict(val_ex) # These are probabilities, and we want to convert them to class labels
label = np.array(prob > 0.5, dtype=int)

# print('The predicted probabilities are: {}'.format(prob))
# print('The predicted class labels are: {}'.format(label))

In [ ]:
# length = len(label)
# print(length)
# b = np.loadtxt('./data/survey_data/uga_2011_locs.txt')
# print(len(b))
# a = b[(512-129):]
# print(len(a))
# for i in range(len(a)):
#     if label[i] == 0:
#         print(a[i], '<default-dot>')
#     elif label[i] == 1:
#         print(a[i], '<green-dot>')
        
c = get_locs()
print(c.shape)

    
    # for i in range(len(b)):
#     if a[i] == 0:
#         print(b[i], '<default-dot>')
#     elif a[i] == 1:
#         print(b[i], '<green-dot>')